In [1]:
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Remplacez ceci par votre clé API réelle
API_KEY = '9a58d306f402d400af1cafd8c6152ec9'

# URL de l'API pour récupérer la liste des sports disponibles
SPORTS_URL = 'https://api.the-odds-api.com/v4/sports'

# URL de l'API pour récupérer les cotes par sport sélectionné
ODDS_URL = 'https://api.the-odds-api.com/v4/sports/{sport}/odds'

# Variable globale pour stocker les DataFrames
global_odds_df = pd.DataFrame()

# Fonction pour récupérer la liste des sports depuis l'API
def get_sports_list(api_key):
    params = {'api_key': api_key}
    response = requests.get(SPORTS_URL, params=params)
    if response.status_code == 200:
        sports_data = response.json()
        sports_list = [sport['key'] for sport in sports_data]
        return sports_list
    else:
        print(f'Failed to fetch sports list: status_code {response.status_code}, response body {response.text}')
        return []

def fetch_odds(button):
    global global_odds_df
    
    sport_keys = sport_select.value
    regions = regions_select.value
    markets = markets_dropdown.value
    odds_format = odds_format_dropdown.value
    date_format = date_format_dropdown.value
    api_key = API_KEY
    
    params = {
        'api_key': api_key,
        'regions': ','.join(regions),
        'markets': markets,
        'oddsFormat': odds_format,
        'dateFormat': date_format,
    }

    all_odds = []
    for sport_key in sport_keys:
        url = ODDS_URL.format(sport=sport_key)
        response = requests.get(url, params=params)
        if response.status_code == 200:
            odds_json = response.json()
            all_odds.extend(odds_json)
            print(f'Sport: {sport_key}, Number of events: {len(odds_json)}')
            # Vérifie le quota d'utilisation
            print('Remaining requests:', response.headers['x-requests-remaining'])
            print('Used requests:', response.headers['x-requests-used'])
        else:
            print(f'Failed to fetch odds for {sport_key}: status_code {response.status_code}, response body {response.text}')
    
    # Nettoyage des résultats précédents
    with output:
        clear_output(wait=True)

        if any(sport.startswith('soccer') for sport in sport_keys) and 'h2h' in markets:
            # Préparation des données pour le DataFrame si le sport commence par "soccer" et le marché est "h2h"
            events = []
            for event in all_odds:
                event_id = event['id']
                sport_key = event['sport_key']
                sport_title = event['sport_title']
                commence_time = event['commence_time']
                home_team = event['home_team']
                away_team = event['away_team']

                for bookmaker in event['bookmakers']:
                    bookmaker_key = bookmaker['key']
                    bookmaker_title = bookmaker['title']
                    win_odd = draw_odd = loose_odd = None

                    for market in bookmaker['markets']:
                        if market['key'] == 'h2h':
                            for outcome in market['outcomes']:
                                if outcome['name'] == home_team:
                                    win_odd = outcome['price']
                                elif outcome['name'] == away_team:
                                    loose_odd = outcome['price']
                                elif outcome['name'] == 'Draw':
                                    draw_odd = outcome['price']

                    # Ajouter uniquement si l'événement n'existe pas encore dans events
                    if (event_id, bookmaker_key) not in [(e['Event ID'], e['Bookmaker Key']) for e in events]:
                        events.append({
                            'Event ID': event_id,
                            'Sport Key': sport_key,
                            'Sport Title': sport_title,
                            'Commence Time': commence_time,
                            'Home Team': home_team,
                            'Away Team': away_team,
                            'Bookmaker Key': bookmaker_key,
                            'Bookmaker Title': bookmaker_title,
                            'Win Odd': win_odd,
                            'Draw Odd': draw_odd,
                            'Loose Odd': loose_odd
                        })

            columns = ['Event ID', 'Sport Key', 'Sport Title', 'Commence Time', 'Home Team', 'Away Team',
                       'Bookmaker Key', 'Bookmaker Title', 'Win Odd', 'Draw Odd', 'Loose Odd']
            df = pd.DataFrame(events, columns=columns)
            display(df)
        else:
            # Préparation des données pour les autres sports
            events = {}
            for event in all_odds:
                game_id = event.get('id')
                commence_time = event.get('commence_time')
                home_team = event['home_team']
                away_team = event['away_team']
                for bookmaker in event.get('bookmakers', []):
                    bookmaker_key = bookmaker.get('key')
                    last_update = bookmaker.get('last_update')

                    
                    if (game_id, bookmaker_key) not in events:
                        events[(game_id, bookmaker_key)] = {
                            'game_id': game_id,
                            'commence_time': commence_time,
                            'bookmaker': bookmaker_key,
                            'last_update': last_update,
                            'home_team': home_team,
                            'away_team': away_team,
                            'market': None,
                            'label_1': None,
                            'odd_1': None,
                            'point_1': None,
                            'label_2': None,
                            'odd_2': None,
                            'point_2': None,
                            'odd_draw': None
                        }
                    for market in bookmaker.get('markets', []):
                        for i, outcome in enumerate(market.get('outcomes', [])):
                            if i == 0:
                                # events[(game_id, bookmaker_key)]['home_team'] = outcome.get('home_team')
                                events[(game_id, bookmaker_key)]['label_1'] = outcome.get('name')
                                events[(game_id, bookmaker_key)]['odd_1'] = outcome.get('price')
                                events[(game_id, bookmaker_key)]['point_1'] = outcome.get('point')
                            elif i == 1:
                                # events[(game_id, bookmaker_key)]['away_team'] = outcome.get('away_team')
                                events[(game_id, bookmaker_key)]['label_2'] = outcome.get('name')
                                events[(game_id, bookmaker_key)]['odd_2'] = outcome.get('price')
                                events[(game_id, bookmaker_key)]['point_2'] = outcome.get('point')
                            elif i == 2:
                                events[(game_id, bookmaker_key)]['odd_draw'] = outcome.get('price')

            # Convertir le dictionnaire en liste pour créer le DataFrame
            events_list = list(events.values())
            columns = ['game_id', 'commence_time', 'bookmaker', 'last_update', 'home_team', 'away_team',
                       'market', 'label_1', 'odd_1', 'point_1', 'label_2', 'odd_2', 'point_2', 'odd_draw']
            df = pd.DataFrame(events_list, columns=columns)
            display(df)
    
    # Mise à jour de la variable globale
    global_odds_df = df

# Récupération de la liste des sports disponibles
sports_list = get_sports_list(API_KEY)

# Création de la liste déroulante des sports avec sélection multiple
sport_select = widgets.SelectMultiple(
    options=sports_list,
    description='Choose sports:',
    disabled=False,
)

# Création de la liste déroulante pour les régions avec sélection multiple
regions_select = widgets.SelectMultiple(
    options=['eu', 'uk', 'us', 'au'],
    value=['us'],
    description='Choose regions:',
    disabled=False,
)

# Création de la liste déroulante pour les marchés (markets)
markets_dropdown = widgets.Dropdown(
    options=['h2h', 'spreads', 'totals'],
    value='h2h',
    description='Choose markets:',
    disabled=False,
)

# Création de la liste déroulante pour le format des cotes (odds format)
odds_format_dropdown = widgets.Dropdown(
    options=['decimal', 'american'],
    value='decimal',
    description='Choose odds format:',
    disabled=False,
)

# Création de la liste déroulante pour le format de date
date_format_dropdown = widgets.Dropdown(
    options=['iso', 'unix'],
    value='iso',
    description='Choose date format:',
    disabled=False,
)

# Création du bouton Fetch
fetch_button = widgets.Button(
    description='Fetch',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Fetch odds data',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Attacher l'événement de clic du bouton à la fonction fetch_odds
fetch_button.on_click(fetch_odds)

# Utilisation d'un Output pour encapsuler les widgets et les résultats
output = widgets.Output()

# Afficher les widgets dans le notebook
display(widgets.VBox([sport_select, regions_select, markets_dropdown, odds_format_dropdown, date_format_dropdown, fetch_button, output]))


In [2]:
global_odds_df

""


# SureBet

In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML



def find_arbitrage_opportunities(df):
    arbitrage_opportunities = []

    # Group by event to find the best odds for each outcome
    grouped = df.groupby('Event ID')
    for event_id, group in grouped:
        home_team = group['Home Team'].iloc[0]
        away_team = group['Away Team'].iloc[0]

        best_win_row = group.loc[group['Win Odd'].idxmax()]
        best_draw_row = group.loc[group['Draw Odd'].idxmax()]
        best_loose_row = group.loc[group['Loose Odd'].idxmax()]

        best_win_odd = best_win_row['Win Odd']
        best_draw_odd = best_draw_row['Draw Odd']
        best_loose_odd = best_loose_row['Loose Odd']

        # Calculate the arbitrage condition
        arbitrage_condition = (1 / best_win_odd) + (1 / best_draw_odd) + (1 / best_loose_odd)
        if arbitrage_condition < 1:
            arbitrage_opportunities.append({
                'Event ID': event_id,
                'Home Team': home_team,
                'Away Team': away_team,
                'Best Win Odd': best_win_odd,
                'Win Bookmaker': best_win_row['Bookmaker Title'],
                'Best Draw Odd': best_draw_odd,
                'Draw Bookmaker': best_draw_row['Bookmaker Title'],
                'Best Loose Odd': best_loose_odd,
                'Loose Bookmaker': best_loose_row['Bookmaker Title'],
                'Arbitrage Condition': arbitrage_condition
            })

    return pd.DataFrame(arbitrage_opportunities)

# Trouver les opportunités d'arbitrage
arbitrage_df = find_arbitrage_opportunities(global_odds_df)

# Widget pour entrer le capital
capital_input = widgets.FloatText(
    description='Capital:',
    value=100.0,
)

# Bouton pour calculer la répartition du capital
calc_button = widgets.Button(
    description='Calculate Investment',
    button_style='success',
)

output = widgets.Output()

def calculate_investment(button):
    capital = capital_input.value
    with output:
        clear_output()
        for index, row in arbitrage_df.iterrows():
            best_win_odd = row['Best Win Odd']
            best_draw_odd = row['Best Draw Odd']
            best_loose_odd = row['Best Loose Odd']

            win_bookmaker = row['Win Bookmaker']
            draw_bookmaker = row['Draw Bookmaker']
            loose_bookmaker = row['Loose Bookmaker']

            # Calculer les investissements
            total_investment = capital * (1 / best_win_odd + 1 / best_draw_odd + 1 / best_loose_odd)
            investment_win = capital / (best_win_odd * (1 / best_win_odd + 1 / best_draw_odd + 1 / best_loose_odd))
            investment_draw = capital / (best_draw_odd * (1 / best_win_odd + 1 / best_draw_odd + 1 / best_loose_odd))
            investment_loose = capital / (best_loose_odd * (1 / best_win_odd + 1 / best_draw_odd + 1 / best_loose_odd))

            # Afficher les résultats
            display(HTML(f"<b>Event ID:</b> {row['Event ID']}, {row['Home Team']} vs {row['Away Team']}"))
            display(HTML(f"Invest on Home Win: {investment_win:.2f} (Odds: {best_win_odd}) at {win_bookmaker}"))
            display(HTML(f"Invest on Draw: {investment_draw:.2f} (Odds: {best_draw_odd}) at {draw_bookmaker}"))
            display(HTML(f"Invest on Away Win: {investment_loose:.2f} (Odds: {best_loose_odd}) at {loose_bookmaker}"))
            display(HTML(f"Total Investment: {investment_win + investment_draw + investment_loose:.2f}"))
            display(HTML(f"Expected Return: {capital-(capital * ((1 / best_win_odd) + (1 / best_draw_odd) + (1 / best_loose_odd))):.2f}"))
            display(HTML("<br>"))

calc_button.on_click(calculate_investment)

# Affichage des widgets
display(widgets.VBox([capital_input, calc_button, output]))

# Afficher les opportunités d'arbitrage pour vérifier
display(arbitrage_df)


KeyError: 'Event ID'

# ValueBet

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML


def find_valuebet_opportunities(df):
    valuebet_opportunities = []

    # Group by event to find the odds for each outcome
    grouped = df.groupby('Event ID')
    for event_id, group in grouped:
        home_team = group['Home Team'].iloc[0]
        away_team = group['Away Team'].iloc[0]

        for outcome in ['Win', 'Draw', 'Lose']:
            outcome_odd_col = f'{outcome} Odd'
            bookmaker_col = 'Bookmaker Title'

            if outcome_odd_col in group.columns:  # Check if the outcome odd column exists
                # Calculate implied probability
                group['Implied Probability'] = 1 / group[outcome_odd_col]

                # Find minimum implied probability (most favorable odds) and corresponding bookmaker
                min_implied_prob = group['Implied Probability'].min()
                max_bookmaker = group.loc[group['Implied Probability'].idxmin(), bookmaker_col]

                # Compare implied probability with estimated probability
                group['Estimated Probability'] = 1 / group[outcome_odd_col]
                group['Value'] = group['Estimated Probability'] - min_implied_prob

                # Find Value Bet opportunities where Value > 0
                valuebet_rows = group[group['Value'] > 0]
                for index, row in valuebet_rows.iterrows():
                    valuebet_opportunities.append({
                        'Event ID': event_id,
                        'Home Team': home_team,
                        'Away Team': away_team,
                        'Outcome': outcome,
                        'Bookmaker': row[bookmaker_col],  # Use .get() to handle missing keys
                        'Odds': row[outcome_odd_col],
                        'Implied Probability': row['Implied Probability'],
                        'Estimated Probability': row['Estimated Probability'],
                        'Value': row['Value']
                    })

    return pd.DataFrame(valuebet_opportunities)

# Exécuter la fonction pour trouver les opportunités de Value Bet
try:
    valuebet_df = find_valuebet_opportunities(global_odds_df)
except Exception as e:
    print(f"Error occurred: {str(e)}")

# Affichage des résultats ou des messages de débogage
if not valuebet_df.empty:
    display(valuebet_df.head())  # Afficher les premières lignes du DataFrame pour vérification
else:
    print("No value bet opportunities found.")

# Afficher les données globales pour vérifier
display(global_odds_df)


,Event ID,Home Team,Away Team,Outcome,Bookmaker,Odds,Implied Probability,Estimated Probability,Value
0,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,Unibet,1.38,0.724638,0.724638,0.010352
1,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,LiveScore Bet (EU),1.32,0.757576,0.757576,0.043290
2,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,Betclic,1.37,0.729927,0.729927,0.015641
3,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,Marathon Bet,1.37,0.729927,0.729927,0.015641
4,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,888sport,1.36,0.735294,0.735294,0.021008


,Event ID,Sport Key,Sport Title,Commence Time,Home Team,Away Team,Bookmaker Key,Bookmaker Title,Win Odd,Draw Odd,Loose Odd
0,7da788bf0563a8cb81a7d980d08c5e18,soccer_uefa_european_championship,UEFA Euro 2024,2024-06-29T16:00:00Z,Switzerland,Italy,unibet_eu,Unibet,3.50,2.90,2.50
1,7da788bf0563a8cb81a7d980d08c5e18,soccer_uefa_european_championship,UEFA Euro 2024,2024-06-29T16:00:00Z,Switzerland,Italy,livescorebet_eu,LiveScore Bet (EU),3.40,2.80,2.43
2,7da788bf0563a8cb81a7d980d08c5e18,soccer_uefa_european_championship,UEFA Euro 2024,2024-06-29T16:00:00Z,Switzerland,Italy,marathonbet,Marathon Bet,3.40,2.94,2.52
3,7da788bf0563a8cb81a7d980d08c5e18,soccer_uefa_european_championship,UEFA Euro 2024,2024-06-29T16:00:00Z,Switzerland,Italy,betclic,Betclic,3.40,2.90,2.50
4,7da788bf0563a8cb81a7d980d08c5e18,soccer_uefa_european_championship,UEFA Euro 2024,2024-06-29T16:00:00Z,Switzerland,Italy,coolbet,Coolbet,3.45,2.90,2.60
...,...,...,...,...,...,...,...,...,...,...,...
123,92d3fec1b1e0a3d2295461f51e8c2a66,soccer_uefa_european_championship,UEFA Euro 2024,2024-07-02T19:00:00Z,Austria,Turkey,matchbook,Matchbook,1.99,3.50,4.70
124,92d3fec1b1e0a3d2295461f51e8c2a66,soccer_uefa_european_championship,UEFA Euro 2024,2024-07-02T19:00:00Z,Austria,Turkey,betsson,Betsson,1.92,3.40,4.40
125,92d3fec1b1e0a3d2295461f51e8c2a66,soccer_uefa_european_championship,UEFA Euro 2024,2024-07-02T19:00:00Z,Austria,Turkey,nordicbet,Nordic Bet,1.90,3.45,4.35
126,92d3fec1b1e0a3d2295461f51e8c2a66,soccer_uefa_european_championship,UEFA Euro 2024,2024-07-02T19:00:00Z,Austria,Turkey,betonlineag,BetOnline.ag,1.93,3.32,4.65


In [ ]:
valuebet_df[valuebet_df['Odds'] < 1.3]


,Event ID,Home Team,Away Team,Outcome,Bookmaker,Odds,Implied Probability,Estimated Probability,Value
109,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,Unibet,1.20,0.833333,0.833333,0.020325
110,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,LiveScore Bet (EU),1.17,0.854701,0.854701,0.041693
111,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,Betclic,1.21,0.826446,0.826446,0.013438
112,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,Marathon Bet,1.21,0.826446,0.826446,0.013438
113,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,William Hill,1.20,0.833333,0.833333,0.020325
114,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,888sport,1.18,0.847458,0.847458,0.034449
115,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,Coolbet,1.22,0.819672,0.819672,0.006664
116,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,Nordic Bet,1.19,0.840336,0.840336,0.027328
117,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,Betsson,1.20,0.833333,0.833333,0.020325
118,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,MyBookie.ag,1.20,0.833333,0.833333,0.020325


# Bookmaker above average


In [7]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

def find_bookmakers_above_average(df):
    valuebet_opportunities = []

    # Group by event to find the odds for each outcome
    grouped = df.groupby('Event ID')
    for event_id, group in grouped:
        home_team = group['Home Team'].iloc[0]
        away_team = group['Away Team'].iloc[0]

        for outcome in ['Win', 'Draw', 'Lose']:
            outcome_odd_col = f'{outcome} Odd'
            bookmaker_col = 'Bookmaker Title'

            if outcome_odd_col in group.columns:  # Check if the outcome odd column exists
                # Calculate the average odd for the event and outcome
                avg_odd = group[outcome_odd_col].mean()

                # Filter bookmakers with odds higher than the average
                higher_than_avg = group[group[outcome_odd_col] > avg_odd]

                for index, row in higher_than_avg.iterrows():
                    valuebet_opportunities.append({
                        'Event ID': event_id,
                        'Home Team': home_team,
                        'Away Team': away_team,
                        'Outcome': outcome,
                        'Bookmaker': row[bookmaker_col],
                        'Odds': row[outcome_odd_col],
                        'Average Odds': avg_odd,
                        'Difference': row[outcome_odd_col] - avg_odd
                    })

    # Create DataFrame from the results and sort by 'Difference' in descending order
    valuebet_opportunities_df = pd.DataFrame(valuebet_opportunities)
    valuebet_opportunities_df.sort_values(by='Difference', ascending=False, inplace=True)

    return pd.DataFrame(valuebet_opportunities_df)

# Exécuter la fonction pour trouver les bookmakers avec des cotes supérieures à la moyenne
try:
    global_odds_df = pd.DataFrame()  # Assurez-vous de définir global_odds_df avec vos données
    above_avg_odds_df = find_bookmakers_above_average(global_odds_df)
except Exception as e:
    print(f"Error occurred: {str(e)}")

# Fonction de filtrage et d'affichage
def filter_odds(change):
    clear_output(wait=True)  # Effacer la sortie précédente
    threshold = float(threshold_widget.value)
    filtered_df = above_avg_odds_df[above_avg_odds_df['Odds'] < threshold]
    display(threshold_widget, apply_button)  # Réafficher les widgets
    display(filtered_df)  # Afficher les résultats filtrés
    return filtered_df  # Retourner le DataFrame filtré

# Widget de saisie pour le seuil de cote
threshold_widget = widgets.FloatText(
    value=1.4,
    description='Threshold:',
    disabled=False
)

# Bouton pour appliquer le filtre
apply_button = widgets.Button(
    description='Apply Filter',
    disabled=False,
    button_style='',
    tooltip='Click to apply the filter',
    icon='check'
)

# Liaison de l'événement de clic du bouton
apply_button.on_click(lambda change: filter_odds(change))

# Affichage des widgets
display(threshold_widget, apply_button)

# Exemple de génération de DataFrame à partir des résultats filtrés
filtered_df = filter_odds(None)
filtered_df.head()  # Afficher les premières lignes du DataFrame filtré


FloatText(value=1.4, description='Threshold:')

Button(description='Apply Filter', icon='check', style=ButtonStyle(), tooltip='Click to apply the filter')

,Event ID,Home Team,Away Team,Outcome,Bookmaker,Odds,Average Odds,Difference
0,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,DraftKings,1.38,1.364444,0.015556
1,3d35b1dc66bc306284408b9f2009ad43,Portugal,Slovenia,Win,BetOnline.ag,1.38,1.364444,0.015556
27,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,SuperBook,1.21,1.194444,0.015556
31,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,BetOnline.ag,1.21,1.194444,0.015556
29,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,MyBookie.ag,1.20,1.194444,0.005556
30,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,DraftKings,1.20,1.194444,0.005556
28,d6fc55a7ce9c0b895191bae5f91019a6,Spain,Georgia,Win,BetMGM,1.20,1.194444,0.005556


In [8]:
filtered_df.to_excel('df.xlsx')